# Исследование NPS

Цель исследования:
        Требовалось определить текущий уровень потребительской лояльности(NPS) на основании предоставленных данных опросов пользователей телекоммуникационной компании. Так же требовалось оценить:
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

# Выгрузка данных

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT 
    user_id --Идентификатор клиента
    ,lt_day --Количество дней «жизни» клиента
    ,case when lt_day<365 then 1 else 0 end is_new --Поле хранит информацию о том, является ли клиент новым
    ,age --Возраст
    ,gender_segment --Пол (для удобства работы с полем преобразуйте значения в текстовый вид)
    ,os_name --Тип операционной системы
    ,cpe_type_name --Тип устройства
    ,country --Страна проживания
    ,city --Город проживания
    ,ags.title as age_segment --Возрастной сегмент
    ,ts.title as traffic_segment --Сегмент по объёму потребляемого трафика
    ,ls.title as lifetime_segment --Сегмент по количеству дней «жизни»
    ,nps_score --Оценка клиента в NPS-опросе
    ,case when nps_score>=9 then 'promoters' 
        when nps_score in (7,8) then 'passives'
        when nps_score<7 then 'detractors'
        else Null end as nps_group --Поле хранит информацию о том, к какой группе относится оценка клиента в опросе
FROM user u
    join location l on u.location_id = l.location_id
    join age_segment ags on ags.age_gr_id = u.age_gr_id
    join traffic_segment ts on ts.tr_gr_id = u.tr_gr_id
    join lifetime_segment ls on ls.lt_gr_id = u.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,0,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,0,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Итоги исследования

Ссылка на отчет в tableau

https://public.tableau.com/views/NPS_16946514345350/NPS?:language=en-GB&publish=yes&:display_count=n&:origin=viz_share_link

Все выводы по исследованию содержаться в нижней части каждого из листов презентации в tableau